In [11]:
import torch,os
from pathlib import Path
import numpy as np
import pandas as pd
import pprint as pp
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 140
import seaborn as sns
from collections import OrderedDict
import pickle,os,json,operator

import utils as utils
import data as data
from importlib import reload
reload(utils);
reload(data);

from sklearn.feature_extraction.text import CountVectorizer
from yellowbrick.text import FreqDistVisualizer

In [14]:
dataset_path='/scratch/ec2684/research/consistency-lm/training_data/wikitext2-sentencized.json'

lm_path = [
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=0/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=1/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=2/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=3/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=4/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=5/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=6/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=7/wikitextv2',
]

strlm_path = [
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.01/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.001/wikitextv2',
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.0001/wikitextv2',
]

osl_path = [
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/test_run/wikitextv2/_235',
]

focal = [
    '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/test_run/wikitextv2/focal_gamma4_lr_0.005_lm',
]

In [15]:
def load_model(model_load_dir):
    if not os.path.exists(model_load_dir):
        raise EOFError("Invalid model load path %s" % model_load_dir)
        
    current_device = "cuda" if torch.cuda.device_count() > 0 else "cpu"
    
    model_dict = torch.load(os.path.join(model_load_dir, "model_best.pt"), map_location=torch.device(current_device))
    return model_dict

In [7]:
def svd(model_path):
    model = load_model(model_path)
    return torch.linalg.svd(model['model_dict']['projection.weight'], full_matrices=True)

In [12]:
osl_svd=svd(osl_path[0])

lm0_svd=svd(lm_path[0])
lm1_svd=svd(lm_path[1])
lm2_svd=svd(lm_path[2])
# lm3_svd=svd(lm_path[3])
# lm4_svd=svd(lm_path[4])

lm3_svd=svd(lm_path[3])
lm4_svd=svd(lm_path[4])

lm5_svd=svd(lm_path[5])
lm6_svd=svd(lm_path[6])
lm7_svd=svd(lm_path[7])

strlm01_svd=svd(strlm_path[0])
strlm001_svd=svd(strlm_path[1])
strlm0001_svd=svd(strlm_path[2])

strlm_svd=torch.linalg.svd(strlm['model_dict']['projection.weight'], full_matrices=True)

In [47]:
# import warnings
# warnings.filterwarnings("ignore")
# fig, ax = plt.subplots(figsize=(10,5))
# sns.distplot(lm_svd[1],ax=ax)
# sns.distplot(strlm_svd[1],ax=ax)
# plt.legend(labels=['lm','strlm'])
# ax.set_ylabel('singular values')
# ax.set_xlabel('word embedding dim.')
# ax.set_title('word embedding of LM and STRLM')

In [11]:
U,S,Vh = osl_svd
cumsum_S=torch.cumsum(S,dim=-1)
print(torch.sum(S))
print(cumsum_S[2]/cumsum_S[-1])

tensor(83354.2656)
tensor(0.0444)


In [30]:
def proj(svd):
    U,S,Vh = svd
    proj = U[:,:2]@torch.diag(S[:2])@Vh[:2, :2]
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,7))
    ax1.scatter(proj[:,0], proj[:,1],alpha=0.5)
    ax2.scatter(proj[:-1,0], proj[:-1,1],alpha=0.5,label='non-eos')
    ax2.scatter(proj[-1,0], proj[-1,1],alpha=1.0,label='<eos>')
    ax2.legend(loc="upper right")
    ax1.axhline(y=0, color='r',linestyle='--')
    ax1.axvline(x=0, color='r',linestyle='--')
    ax2.axhline(y=0, color='r',linestyle='--')
    ax2.axvline(x=0, color='r',linestyle='--')
    plt.show()

In [31]:
lm_file_0 = '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_tied_seeds/0108_1930/wikitext2_sentencized_tied_seeds____seed=0/wikitextv2/eval/eval.pkl'
st_file_1E_minus_2 = '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.01/wikitextv2/eval/eval.pkl'
st_file_1E_minus_3 = '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.001/wikitextv2/eval/eval.pkl'
st_file_1E_minus_4 = '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.0001/wikitextv2/eval/eval.pkl'

st_finetuned_1E_minus_4 ='/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.0001_finetune/wikitextv2/_6/eval/eval.pkl'

st_finetuned_untied_1E_minus_4 ='/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.0001_finetune/wikitextv2/_12/eval/eval.pkl'

# st_finetuned_1E_minus_4_w_osl='/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.0001_pretrain_finetune/wikitextv2/_109/eval/eval.pkl'

st_finetuned_1E_minus_4_w_osl_65 = '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.0001_pretrain_finetune/65/wikitextv2/eval/eval.pkl'
st_finetuned_1E_minus_4_w_osl_99 = '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.0001_pretrain_finetune/99/wikitextv2/eval/eval.pkl'
st_finetuned_1E_minus_4_w_osl_95 = '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/0108_1954/wikitext2_sentencized_st_lstm____st-epsilon=0.0001_pretrain_finetune/95/wikitextv2/eval/eval.pkl'

focal = '/scratch/ec2684/research/consistency-lm/runs/wikitext2_sentencized_st_lstm/test_run/wikitextv2/_266/eval/eval.pkl'

In [32]:
file_dirs = {
    'l.m.': {
        '0': lm_file_0,
    },
    's.t.': {
        '1E-2':st_file_1E_minus_2,
        '1E-3':st_file_1E_minus_3,
        '1E-4':st_file_1E_minus_4,
    },
    'lm_st': {
        '1E-4':st_finetuned_1E_minus_4,
    },
    'lm_st_untied': {
        '1E-4':st_finetuned_untied_1E_minus_4,
    },
    'lm_st_osl': {
        '0.65':st_finetuned_1E_minus_4_w_osl_65,
        '0.95':st_finetuned_1E_minus_4_w_osl_95,
        '0.99':st_finetuned_1E_minus_4_w_osl_99,
    },
    'focal': {
        '4':focal,
    },
}

In [33]:
def get_data(file_dir: dict):
    data = {}
    for key in sorted(file_dir.keys()):
        if isinstance(file_dir[key], str):
            with open(file_dir[key], 'rb') as f:
                data[key] = pickle.load(f)
        else:
            data[key] = get_data(file_dir[key])
    return data

In [34]:
data = get_data(file_dirs)

In [35]:
dataset = 'train'

In [36]:
def get_prefix(model: str, setting: str):
    return data[model][setting][dataset]['greedy']['decoding_logs'][0]

def get_continuation(model: str, setting: str):
    return data[model][setting][dataset]['greedy']['decoding_logs'][1]

def get_target(setting: str):
    return data['s.t.'][setting][dataset]['greedy']['decoding_logs'][2]

In [37]:
assert all(get_prefix('s.t.','1E-2') == get_prefix('s.t.',key) for key in data['s.t.'].keys())
assert all(get_prefix('s.t.','1E-2') == get_prefix('l.m.',key) for key in data['l.m.'].keys())

In [38]:
# def get_continuation(model: str, setting: str, idx: int):
#     return " ".join(get_continuation(model, setting)[idx])
def get_cont_dict(model: str, idx: int):
    continuations = {}
    for key in data[model].keys():
        continuations[key] = get_continuation(model,key)[idx]
    return continuations

In [39]:
non_terminated = ['a','b','c','c']

In [40]:
len(set(non_terminated))/ max(len(non_terminated), 1),

(0.75,)

In [42]:
for key in data.keys():
    print(f'\n\nModel type: {key}')
    for key1 in data[key].keys():
        print(f'- setting: {key1}')
        for key2 in data[key][key1]:
            print(f'{" "*2}- dataset: {key2}')
            for key3 in data[key][key1][key2]:
                if key2 == 'ppl':
                    # print(f'{" "*4}- {key3}')
                    print(f'{" "*4}- {key3}:\t{" "*6}{data[key][key1][key2][key3]}')
                else: 
                    # if key3 != 'greedy':
                        # continue
                    print(f'{" "*4}- decoding algo.: {key3}')
                    for key4 in data[key][key1][key2][key3]:
                        if key4 == 'decoding_logs':
                            continue
                        elif key4 == 'uniq_nonterminated':
                            print(f'{" "*6}- {key4}:\t{data[key][key1][key2][key3][key4]}')
                        else:
                            print(f'{" "*6}- {key4}:\t\t{data[key][key1][key2][key3][key4]}')
                    print()



Model type: focal
- setting: 4
  - dataset: train
    - decoding algo.: beam_2
      - nonterminated:		0.0
      - uniq_nonterminated:	0.0
      - avg_len:		6.875496031746032

    - decoding algo.: beam_4
      - nonterminated:		0.0
      - uniq_nonterminated:	0.0
      - avg_len:		3.7604166666666665

    - decoding algo.: greedy
      - nonterminated:		0.0
      - uniq_nonterminated:	0.0
      - avg_len:		14.260912698412698

    - decoding algo.: sample
      - nonterminated:		0.0
      - uniq_nonterminated:	0.0
      - avg_len:		15.324404761904763

    - decoding algo.: topk-2
      - nonterminated:		0.0
      - uniq_nonterminated:	0.0
      - avg_len:		12.37202380952381

    - decoding algo.: topk-4
      - nonterminated:		0.0
      - uniq_nonterminated:	0.0
      - avg_len:		11.935515873015873

    - decoding algo.: topp-0.2
      - nonterminated:		0.000496031746031746
      - uniq_nonterminated:	1.0
      - avg_len:		13.610615079365079

    - decoding algo.: topp-0.4
      - non

In [48]:
length_check = lambda data, length: any(len(data[key]) > length for key in data.keys())
length = 30

for idx in range(1000):
    st_cont = get_cont_dict('s.t.',idx)
    lm_cont = get_cont_dict('l.m.',idx)
    lm_st_cont = get_cont_dict('lm_st',idx)
    lm_st_untied_cont = get_cont_dict('lm_st_untied',idx)
    lm_st_osl_cont = get_cont_dict('lm_st_osl',idx)
    focal_cont = get_cont_dict('focal',idx)
    # if length_check(st_cont, length) or length_check(st_cont, length) or length_check(lm_st_cont, length):
    if length_check(st_cont, length) or length_check(st_cont, length) or length_check(lm_st_cont, length):
        print(f'\n\nIndex {idx}:')
        print(f'''(origin): {" ".join(get_target('1E-4')[idx])}''')
        print(f'''(prefix): {" ".join(get_prefix('l.m.','0')[idx][1:])}''')
        print(f'''(target):   {" ".join(get_target('1E-3')[idx][len(get_prefix('l.m.','0')[idx][1:]):])}''')
        print(f'\n(self-terminating):')
        for key in st_cont.keys():
            # print(f'''($\\varepsilon$ = {key}): {" ".join(st_cont[key])}''')
            print(f'''(ε = {key}): {" ".join(st_cont[key])}''')
        print(f'\n(language model):')
        for key in lm_cont.keys():
            print(f'''(seed = {key}): {" ".join(lm_cont[key])}''')
        print(f'\n(pretrain w/ language model & fine-tune w/ self-terminating):')
        for key in lm_st_cont.keys():
            print(f'''(ε = {key}): {" ".join(lm_st_cont[key])}''')
        print(f'\n(pretrain w/ language model & fine-tune w/ self-terminating) w/ untied word embedding:')
        for key in lm_st_untied_cont.keys():
            print(f'''(ε = {key}): {" ".join(lm_st_untied_cont[key])}''')
        print(f'\n(pretrain w/ language model & fine-tune w/ self-terminating and osl):')
        for key in lm_st_osl_cont.keys():
            print(f'''(α = {key}): {" ".join(lm_st_osl_cont[key])}''')
        print(f'\n(focal):')
        for key in focal_cont.keys():
            print(f'''(γ = {key}): {" ".join(focal_cont[key])}''')



Index 1:
(origin): Released in January 2011 in Japan , it is the third game in the Valkyria series .
(prefix): Released in January 2011 in Japan , it is the
(target):   third game in the Valkyria series .

(self-terminating):
(ε = 1E-2): first episode of the series , and the second episode of the series ,
(ε = 1E-3): first episode of the first season of the American television series The Office of the Year .
(ε = 1E-4): first song to be released in the United States on November 30 , 2003 .

(language model):
(seed = 0): first song to be released in the United States .

(pretrain w/ language model & fine-tune w/ self-terminating):
(ε = 1E-4): first episode of the series , and the second episode of the series , " The <unk> of the Palisades " , was released in the United States on November 18 , 2012 .

(pretrain w/ language model & fine-tune w/ self-terminating) w/ untied word embedding:
(ε = 1E-4): first episode of the series to be released in the United States .

(pretrain w/ language

In [33]:
lm_cont

{'0': ['she', 'is', 'a', '<unk>', 'of', 'the', 'family', '.']}